# Extracción de palabras frecuentes en un .csv

- Palabras y frases de la base oculta con palabras claves elegidas con ruralidad
- Extracción de listado único de organizaciones criminales en la base


Proyecto Conflicto armado Mx

In [3]:
#!pip3 install nltk
# En este caso instalamos desde el entorno externo

# Se usa con la base de datos oculta: https://concopiaoculta.org/la-bitacora-de-la-guerra/base-de-datos

In [5]:
import csv
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Descargar datos necesarios para la tokenización (solo se necesita hacerlo una vez)
nltk.download('punkt')

# Ruta al archivo CSV
csv_file = "/home/omnibus/Downloads/AAA-Trabajo-mayo-16-Convocatorias/Chamba-Violencia-MX/base_oculta.csv"

# Lista para almacenar todas las palabras del campo "texto"
palabras = []


# Leer el archivo CSV y extraer el contenido del campo "texto"
with open(csv_file, 'r', newline='', encoding='iso-8859-1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        texto = row['descripcion']
        # Tokenizar las frases en palabras
        frases = sent_tokenize(texto)
        for frase in frases:
            palabras.extend(word_tokenize(frase))

# Contar la frecuencia de cada palabra
frecuencia_palabras = Counter(palabras)

# Imprimir las palabras más frecuentes
print("Palabras más frecuentes:")
for palabra, frecuencia in frecuencia_palabras.most_common(10):  # Las 10 palabras más frecuentes
    print(f"{palabra}: {frecuencia}")


[nltk_data] Downloading package punkt to /home/omnibus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Palabras más frecuentes:
de: 130644
en: 82698
]: 69352
[: 69344
,: 47302
la: 45050
ubicacion: 42867
.: 40596
descripcion: 35977
proceso: 35974


In [ ]:
## Sin Stop Words

In [6]:
import csv
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string

# Descargar datos necesarios para la tokenización y las stop words (solo necesitas hacerlo una vez)
nltk.download('punkt')
nltk.download('stopwords')

# Ruta al archivo CSV
csv_file = "/home/omnibus/Downloads/AAA-Trabajo-mayo-16-Convocatorias/Chamba-Violencia-MX/base_oculta.csv"

# Lista para almacenar todas las palabras del campo "texto"
palabras = []

# Cargar lista de stop words en español
stop_words = set(stopwords.words('spanish'))

# Leer el archivo CSV y extraer el contenido del campo "texto"
with open(csv_file, 'r', newline='', encoding='iso-8859-1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        texto = row['descripcion']
        # Tokenizar las frases en palabras
        frases = sent_tokenize(texto)
        for frase in frases:
            # Tokenizar palabras y eliminar puntuación
            palabras_frase = word_tokenize(frase)
            palabras_filtradas = [palabra.lower() for palabra in palabras_frase if palabra.lower() not in stop_words and palabra.lower() not in string.punctuation]
            palabras.extend(palabras_filtradas)

# Contar la frecuencia de cada palabra
frecuencia_palabras = Counter(palabras)

# Imprimir las palabras más frecuentes (excluyendo stop words y puntuación)
print("Palabras más frecuentes:")
for palabra, frecuencia in frecuencia_palabras.most_common(10):  # Las 10 palabras más frecuentes
    print(f"{palabra}: {frecuencia}")


[nltk_data] Downloading package punkt to /home/omnibus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/omnibus/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Palabras más frecuentes:
ubicacion: 42867
descripcion: 35977
proceso: 35974
anonimizacion: 35970
nombre: 14724
arma: 12782
colonia: 12385
fuego: 12296
impactos: 9801
edad: 9780


## Buscando de un listado específico de palabras clave relacionadas con el entorno rural

In [2]:
import csv
import re
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets

# Ruta al archivo CSV
csv_file = "/home/omnibus/Downloads/AAA-Trabajo-mayo-16-Convocatorias/Chamba-Violencia-MX/base_oculta.csv"

# Lista de palabras clave relacionadas con entorno rural o población rural
palabras_clave = ['rural', 'campo', 'agricultura', 'población rural', 'zonas rurales', 'comunidad rural']

# Lista para almacenar las frases que contienen palabras clave
frases_con_informacion_rural = []

# Leer el archivo CSV y buscar frases con palabras clave
with open(csv_file, 'r', newline='', encoding='iso-8859-1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        texto = row['descripcion']
        # Buscar frases que contengan palabras clave (ignorando mayúsculas y minúsculas)
        frases = re.split(r'[.!?]', texto)  # Separar el texto en frases
        frases_con_palabras_clave = [frase.strip() for frase in frases if any(palabra.lower() in frase.lower() for palabra in palabras_clave)]
        frases_con_informacion_rural.extend(frases_con_palabras_clave)

# Crear un DataFrame con las frases encontradas
df = pd.DataFrame(enumerate(frases_con_informacion_rural, start=1), columns=['N°', 'Frases con información rural'])

# Crear una tabla HTML a partir del DataFrame
tabla_html = df.to_html(index=False)

# Mostrar la tabla HTML en el notebook de Jupyter
display(HTML(tabla_html))

# Función para descargar archivos en formato CSV, MD y HTML
def download_data(formato):
    if formato == "csv":
        df.to_csv("frases_con_informacion_rural.csv", index=False)
    elif formato == "md":
        with open("frases_con_informacion_rural.md", "w") as mdfile:
            mdfile.write(tabla_html)
    elif formato == "html":
        with open("frases_con_informacion_rural.html", "w") as htmlfile:
            htmlfile.write(tabla_html)
    else:
        print("Formato inválido. No se realizará ninguna descarga.")

# Crear una lista desplegable con los tipos de formato
formatos_dropdown = widgets.Dropdown(options=["csv", "md", "html"], value="csv", description="Formato:")

# Crear un botón de descarga
descargar_boton = widgets.Button(description="Descargar")

# Función para manejar la descarga cuando se presione el botón
def on_descargar_boton_click(boton):
    formato_elegido = formatos_dropdown.value
    download_data(formato_elegido)

# Asignar la función al evento "click" del botón
descargar_boton.on_click(on_descargar_boton_click)

# Mostrar el botón y la lista desplegable en el notebook de Jupyter
display(formatos_dropdown, descargar_boton)


N°,Frases con información rural
1,en su domicilio ubicado en calle [ubicacion] de la colonia [ubicacion] sujetos armados intentaron privar de la libertad al policia rural [nombre] quien privo de la vida a uno de los presuntos delincuentes
2,"en la carretera [ubicacion] cerca de los campos [ubicacion], fue localizado un cadaver masculino atado de pies y manos"
3,en un paraje de terraceria que conduce a la comunidad de [ubicacion] en el campo conocido como [ubicacion] se localizo el cuerpo desmembrado de una persona
4,"en una zona rural del camino a [ubicacion] fue localizado el cuerpo de un masculino, el cual presentaba impactos por arma de fuego en la cabeza"
5,en la brecha que comunica a la comunidad rural [ubicacion] fue localizado el cuerpo de un individuo calcinado y amordazado
6,"en la entrada al campo [ubicacion], sindicatura de [ubicacion] fue encontrado el cuerpo de un sujeto que presento multiples impactos de bala 9 mm, ademas estaba envuelto en una cobija"
7,"en el lugar conocido como [ubicacion], ubicado junto al campo de beisbol, sujetos interceptaron a [nombre] y/o [nombre] (a) [nombre] ([edad] aÃ±os) y [nombre] ([edad] aÃ±os), quienes resultaron lesionados"
8,en el campo [ubicacion] de la localidad [ubicacion] fue asesinado [nombre] ([edad] aÃ±os) y lesionada [nombre] con impactos de arma de fuego
9,a un costado del campo de futbol de la colonia [ubicacion] un hombre fue ultimado con ocho impactos de arma de fuego
10,"en un campo de futbol, ubicado en [ubicacion], fue localizado el cuerpo de [nombre] [nombre] el chicha con un impacto de arma de fuego en el lado derecho de la cabeza"


Dropdown(description='Formato:', options=('csv', 'md', 'html'), value='csv')

Button(description='Descargar', style=ButtonStyle())

In [1]:
import csv
import re
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets
import random

# Ruta al archivo CSV
csv_file = "/home/omnibus/Downloads/AAA-Trabajo-mayo-16-Convocatorias/Chamba-Violencia-MX/base_oculta.csv"

# Lista de palabras clave relacionadas con entorno rural o población rural
palabras_clave = ['comunidad','ejido','rural','agrario','agraria','campesina','agricultor','productor', 'campo', 'agricultura', 'población rural', 'zonas rurales', 'comunidad rural']

# Lista para almacenar las frases que contienen palabras clave
frases_con_informacion_rural = []

# Leer el archivo CSV y buscar frases con palabras clave
with open(csv_file, 'r', newline='', encoding='iso-8859-1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        texto = row['descripcion']
        # Buscar frases que contengan palabras clave (ignorando mayúsculas y minúsculas)
        frases = re.split(r'[.!?]', texto)  # Separar el texto en frases
        frases_con_palabras_clave = [frase.strip() for frase in frases if any(palabra.lower() in frase.lower() for palabra in palabras_clave)]
        frases_con_informacion_rural.extend(frases_con_palabras_clave)

# Crear un DataFrame con las frases encontradas
df = pd.DataFrame(enumerate(frases_con_informacion_rural, start=1), columns=['N°', 'Frases con información rural'])

# Crear una tabla HTML a partir del DataFrame con las palabras clave resaltadas en un color personalizado
def highlight_keywords(row, keywords, color):
    for keyword in keywords:
        row = row.replace(keyword, f'<span style="background-color: {color};">{keyword}</span>')
    return row

# Generar una tabla HTML con las palabras clave resaltadas
def generate_html_table_with_highlight(df, keywords, color):
    df_highlighted = df.copy()
    df_highlighted['Frases con información rural'] = df_highlighted['Frases con información rural'].apply(lambda x: highlight_keywords(x, keywords, color))
    return df_highlighted.to_html(index=False, escape=False)

# Crear una lista desplegable con los tipos de formato
formatos_dropdown = widgets.Dropdown(options=["csv", "md", "html"], value="csv", description="Formato:")

# Crear un widget de selección de color
color_picker = widgets.ColorPicker(value="yellow", description="Color de resaltado:")

# Crear un botón de descarga
descargar_boton = widgets.Button(description="Descargar")

# Función para manejar la descarga cuando se presione el botón
def on_descargar_boton_click(boton):
    formato_elegido = formatos_dropdown.value
    color_elegido = color_picker.value
    if formato_elegido == "csv":
        df.to_csv("frases_con_informacion_rural.csv", index=False)
    elif formato_elegido == "md":
        with open("frases_con_informacion_rural.md", "w") as mdfile:
            mdfile.write(generate_html_table_with_highlight(df, palabras_clave, color_elegido))
    elif formato_elegido == "html":
        with open("frases_con_informacion_rural.html", "w") as htmlfile:
            htmlfile.write(generate_html_table_with_highlight(df, palabras_clave, color_elegido))
    else:
        print("Formato inválido. No se realizará ninguna descarga.")

# Asignar la función al evento "click" del botón
descargar_boton.on_click(on_descargar_boton_click)

# Mostrar el botón, la lista desplegable y el selector de color en el notebook de Jupyter
display(formatos_dropdown, color_picker, descargar_boton)

# Mostrar la tabla HTML con las palabras clave resaltadas
display(HTML(generate_html_table_with_highlight(df, palabras_clave, color_picker.value)))


Dropdown(description='Formato:', options=('csv', 'md', 'html'), value='csv')

ColorPicker(value='yellow', description='Color de resaltado:')

Button(description='Descargar', style=ButtonStyle())

N°,Frases con información rural
1,en la comunidad de [ubicacion] se registro el homicidio de [nombre] ([edad] aÃ±os) y [nombre] ([edad] aÃ±os) por multiples impactos de arma de fuego calibre 7
2,"en la comunidad [ubicacion] sujetos armados privaron de la vida a [nombre] y [nombre] cuando se encontraban en el interior de un domicilio, los agresores prendieron fuego al inmueble"
3,en un domicilio de la comunidad de [ubicacion] hombres armados asesinaron a [nombre] (a) [nombre]
4,en la comunidad [ubicacion] se localizo un hombre con un impacto de bala en la cabeza calibre 9 mm
5,"en tecolutla, a las [hora] horas, en el camino de terracerria que conduce a la congreracion de [ubicacion] aproximadamente a 300 metros del kilometro 067+600, de la carretera [ubicacion] tramo [ubicacion] de a la comunidad [ubicacion] fue localizado el cuerpo del empresario [nombre] ([edad] aÃ±os) con un impacto de bala en la frente"
6,"en la calle [ubicacion] de la comunidad [ubicacion] fueron ultimados [nombre] ([edad] aÃ±os), su [parentesco] [nombre] ([edad] aÃ±os) asi como [nombre] ([edad] aÃ±os) con impactos de bala calibre"
7,en un camino de terraceria que conduce a la comunidad [ubicacion] fue localizado un cuerpo femenino con impactos de bala en la cabeza
8,"en la calle [ubicacion] de la comunidad [ubicacion] fue asesinado con impactos de arma de fuego calibre 9 mm [nombre] ([edad] aÃ±os), por individuos a bordo de una camioneta"
9,en el camino que conduce a la comunidad de [ubicacion] fue ultimado con impactos de bala calibre 556 y 38 super [nombre] quien se encontraba a bordo de una camioneta
10,"en la comunidad [ubicacion] fue encontrado un cuerpo decapitado, en estado de descomposicion y cubierto con una sabana"


## Extraer nombres únicos de organizaciones criminales (BACRIM)

In [1]:
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import FileLink

# Leer el archivo CSV
df = pd.read_csv('/home/omnibus/Documents/QGISS_Cartografia/Proyectos/criminal_mx/BaseCarteles-El-Universal2.csv')

# Obtener la lista de campos presentes en el archivo CSV
campos_disponibles = df.columns.tolist()

# Lista de palabras clave
palabra_clave = 'Cártel'

# Crear la lista desplegable para elegir los campos de búsqueda
campo_dropdown = widgets.SelectMultiple(
    options=campos_disponibles,
    description='Campos:',
)

# Mostrar la lista desplegable para que el usuario seleccione los campos de búsqueda
display(campo_dropdown)

# Función para filtrar los registros según los campos y palabra clave seleccionados
def filtrar_registros(campos, palabra_clave):
    # Convertir los campos seleccionados a tipo str
    campos_str = [str(campo) for campo in campos]
    
    # Filtrar los registros que comienzan con la palabra clave en los campos seleccionados
    df_filtrado = df[df.apply(lambda row: any(str(row[campo]).startswith(palabra_clave) for campo in campos_str), axis=1)].drop_duplicates(subset=campos_str)
    # Filtrar los registros que NO comienzan con la palabra clave en los campos seleccionados
    df_no_filtrado = df[df.apply(lambda row: not any(str(row[campo]).startswith(palabra_clave) for campo in campos_str), axis=1)].drop_duplicates(subset=campos_str)

    # Contar el total de organizaciones únicas en cada tabla
    total_filtrado = df_filtrado.drop_duplicates(subset=campos_str).shape[0]
    total_no_filtrado = df_no_filtrado.drop_duplicates(subset=campos_str).shape[0]
    
    # Obtener el total de organizaciones únicas en el DataFrame original (gran total)
    gran_total = df.drop_duplicates(subset=campos_str).shape[0]

    # Crear una tabla HTML con los DataFrames filtrados y los totales
    tabla_html = f'''
    <h2>Organizaciones que comienzan con "{palabra_clave}" en los campos {", ".join(campos_str)}</h2>
    {df_filtrado.to_html(index=False)}
    <p>Total de organizaciones únicas: {total_filtrado}</p>

    <h2>Organizaciones que NO comienzan con "{palabra_clave}" en los campos {", ".join(campos_str)}</h2>
    {df_no_filtrado.to_html(index=False)}
    <p>Total de organizaciones únicas: {total_no_filtrado}</p>

    <p>Total de organizaciones con nombre Cártel: {total_filtrado}</p>
    <p>Total de organizaciones sin nombre Cártel: {total_no_filtrado}</p>
    <h2>Gran Total (organizaciones únicas en todos los campos)</h2>
    <p>{gran_total}</p>
    '''

    # Mostrar la tabla HTML
    display(HTML(tabla_html))

    # Crear la lista desplegable para elegir el formato de exportación
    formato_dropdown = widgets.Dropdown(
        options=['.md', '.csv', '.html'],
        value='.html',
        description='Formato:',
    )

    # Función para exportar el archivo al oprimir el botón "Descargar"
    def exportar_archivo(formato):
        if formato == '.md':
            df.to_markdown('archivo.md', index=False)
        elif formato == '.csv':
            df.to_csv('archivo.csv', index=False)
        elif formato == '.html':
            with open('archivo.html', 'w') as f:
                f.write(tabla_html)
        display(FileLink(f'archivo{formato}'))

    # Agregar el botón "Descargar"
    descargar_boton = widgets.Button(description="Descargar")
    descargar_boton.on_click(lambda b: exportar_archivo(formato_dropdown.value))
    display(formato_dropdown, descargar_boton)

# Widget interactivo para actualizar la tabla cuando el usuario seleccione campos
widgets.interactive(filtrar_registros, campos=campo_dropdown, palabra_clave=widgets.fixed(palabra_clave))


SelectMultiple(description='Campos:', options=('ClaveEntidad', 'ClaveMunicipio', 'ClaveGeo', 'Municipio', 'Est…

interactive(children=(SelectMultiple(description='Campos:', options=('ClaveEntidad', 'ClaveMunicipio', 'ClaveG…

# Versión sencilla sin filtrado de palabras clave. Sólo unificar, limpiar de repeticiones y ordenar

In [33]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink

# Leer el archivo CSV
df = pd.read_csv('/home/omnibus/Documents/QGISS_Cartografia/Proyectos/criminal_mx/BaseCarteles-El-Universal2.csv')

# Crear la lista desplegable para elegir los campos de búsqueda
campo_dropdown = widgets.SelectMultiple(
    options=df.columns.tolist(),
    description='Campos:',
)
display(campo_dropdown)

# Campo de texto para ingresar palabras clave
palabra_clave_text = widgets.Text(
    placeholder='Ingrese palabras clave separadas por coma',
    description='Palabra clave:',
)
display(palabra_clave_text)

def filtrar_registros(campos, palabra_clave):
    campos_str = [str(campo) for campo in campos]
    filtered_data = df[campos_str]

    keywords = [keyword.strip() for keyword in palabra_clave.split(',')]

    unified_list = []

    for col in campos_str:
        col_data = filtered_data[col].dropna().astype(str)
        keyword_mask = col_data.apply(lambda cell: any(keyword in cell for keyword in keywords))
        col_filtered = col_data[keyword_mask]
        unified_list.extend(col_filtered.tolist())

    # Eliminar duplicados y ordenar alfabéticamente
    unified_list = sorted(list(set(unified_list)))
    num_elementos_unicos = len(unified_list)  # Contar elementos únicos
    
    tabla_html = '<h2>Listado final unificado sin repeticiones (orden alfabético)</h2>'
    tabla_html += '<ul>'
    for item in unified_list:
        tabla_html += f'<li>{item}</li>'
    tabla_html += '</ul>'
    tabla_html += f'''<h2>Total de elementos únicos: {num_elementos_unicos} </h2>'''
          
    display(HTML(tabla_html))

    formato_dropdown = widgets.Dropdown(
        options=['.md', '.csv', '.html'],
        value='.html',
        description='Formato:',
    )

    def exportar_archivo(formato):
        if formato == '.md':
            with open('listado_org.md', 'w') as f:
                f.write('\n'.join(unified_list))
        elif formato == '.csv':
            df_result = pd.DataFrame({'Items': unified_list})
            df_result.to_csv('listado_org.csv', index=False)
        elif formato == '.html':
            with open('listado_org.html', 'w') as f:
                f.write(tabla_html)
        display(FileLink(f'archivo{formato}'))

    descargar_boton = widgets.Button(description="Descargar")
    descargar_boton.on_click(lambda b: exportar_archivo(formato_dropdown.value))
    display(formato_dropdown, descargar_boton)

widgets.interactive(filtrar_registros, campos=campo_dropdown, palabra_clave=palabra_clave_text)


SelectMultiple(description='Campos:', options=('ClaveEntidad', 'ClaveMunicipio', 'ClaveGeo', 'Municipio', 'Est…

Text(value='', description='Palabra clave:', placeholder='Ingrese palabras clave separadas por coma')

interactive(children=(SelectMultiple(description='Campos:', options=('ClaveEntidad', 'ClaveMunicipio', 'ClaveG…